# auto translate session caption

这个notebook主要用来对wwdc中session的字幕进行爬取，然后翻译。起因是因为看视频的时候，需要各种暂停思考，而且视频的质量跟网速也有关系，每个视频下载下来也占地方，还不如直接观看keynote。

一个问题是，除了keynote之外，还有一些点是通过讲出来的，因此将字幕下载下来、翻译，结合这keynote一起观看是目前比较舒服的一种方法。

主调函数在最下面：`scrpit_wwdc_then_translate_save(year,session)`，传入对应的year、session即可完成下载、翻译、保存操作。

## prepare

将一些库进行引入，以及设置全局变量、工具函数等。

In [55]:
import httplib
import hashlib
import urllib
import random
import requests
import sys
import json
import re
import os
from bs4 import BeautifulSoup
import pandas as pd
import time

appid = '20170524000049056'#如果调用次数不够了，可以再申请一个
secretKey = 'mannZwutpWn3BVpUPbdy'#如果调用次数不够了，可以再申请一个

httpClient = None
translate_url = '/api/trans/vip/translate'
fromLang = 'auto'
toLang = 'zh'

USER_AGENTS = [
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0",
                "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
                "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
                "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
                "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
                "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
                "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
                "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
                "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5"]

def mkdir(path):
 
    folder = os.path.exists(path)
 
    if not folder:
        os.makedirs(path)
        print "--- create folder %s --- ",path
    else:
        print "---  There is this folder!  ---"

def file_name(year,session,suffix = 'txt'):
    return 'wwdc{}_{}.{}'.format(year,session,suffix)

def folder_path(year,session):
    return './wwdc/wwdc{}_{}'.format(year,session)
    

## translate

执行百度翻译接口对字幕进行翻译。这一部分没有什么，基本上是参考文档上来进行，需要注意的是由于接口默认返回的是utf-8编码，对中文的打印不是很友好，需要对dst进行一下转换。

In [102]:
def translate(caption):
    
    if caption is None or len(caption) == 0:
        return '',''
    
    time.sleep(1)
    
    salt = random.randint(32768, 65536)

    sign = appid+caption+str(salt)+secretKey
    
    md5 = hashlib.md5()
    md5.update(sign)
    sign_md5 = md5.hexdigest()

    t = urllib.quote(caption)
    
    url = translate_url + '?appid={}&q={}&from={}&to={}&salt={}&sign={}'.format(appid,t,fromLang,toLang,str(salt),sign_md5)
    full_url = 'api.fanyi.baidu.com' + url
    
    try:
        httpClient = httplib.HTTPConnection('api.fanyi.baidu.com')
        httpClient.request('GET', url)

        #response是HTTPResponse对象
        response = httpClient.getresponse()

        result = response.read()
        if result is None :
            return ' ',' '
        
        trans_result = eval(result)['trans_result']# 使用eval函数将字符串转成对应的数组、字典
        src = trans_result[0]['src']
        dst = trans_result[0]['dst']

        return caption,dst.decode('unicode_escape')
    except Exception as e:
        print(e)
    finally:
        if httpClient:
            httpClient.close()

## read caption

从字幕文件中读取文本，如果逐行进行翻译，会有大量的api调用，对于一个免费版本的key来说有点儿浪费，这里使用一个策略是设置x行一起翻译。基本上一个session有300行左右的信息，这里默认为10行，最后也就30次api的调用，再多之后，阅读起来就不是很美观。

In [3]:
def read_caption_content(caption_path,lines = 10):
    
    def read_lines_caption():
        '''
        将每读取10行的数据保存为一个段落
        '''
        sum_caption = []
        while 1:
            lines = file.readlines(100000)

            if not lines:
                break

            for line in lines:
                sum_caption.append(str(line))
    
        return sum_caption
    
    file = open(caption_path)
    captions = read_lines_caption()
    ten_line_captions = []
    
    for i in range(len(captions)/lines + 1):
        f = i * lines
        t = i * lines + lines
        c = ''.join(captions[f:t])
        c = re.sub('[\r\n\t]', '', c)
        ten_line_captions.append(c)

    return ten_line_captions

## translate and caption

根据文件位置进行读取、翻译、保存本地的操作。

In [109]:
def translate_caption(caption_path):
    
    def save_file_path():
        
        paths = caption_path.split('/')
        file_path = 'transcript_' + paths[3]
        paths[3] = file_path

        return '/'.join(paths)
    
    def save_translate(src,res):
        '''
        将文字进行组合，由于有中文的存在，需要对字符进行编码设置，
        在调用api之后的decode操作是unicode，保存的时候必须是utf8
        '''
        text = ' '
        try:
            if src is ' ' or res is ' ' :
                return ' '
            text = src.encode('utf8') + '\n\n'+ res.encode('utf8') + '\n\n'
        except Exception as e:
            print e
            
        return text
    
    captions = read_caption_content(caption_path)
    
    if len(captions) is not 0:

        file = open(save_file_path(),'w')
        index = 1
        for caption in captions:
            
            print 'translating:{}/{}'.format(index,len(captions))
            
            t = translate(caption)
            src = ' '
            res = ' '
            try:
                src = t[0]
                res = t[1]
            except Exception as e:
                print t
                print e
                
            text = save_translate(src,res)
            
            file.write(text) 
            index += 1
            
        file.close()
    

## script wwdc

根据year、session去官网爬取对应的字幕信息，这里选择的是script而不是download，其实使用download也可以达到想要的效果。由于这里是定向爬取的，不会对官网所有的字幕进行爬取，所以没有进行anit-anit-script操作。

In [87]:
def scrpit_wwdc(year,session):
    
    folder = folder_path(year,session)
    mkdir(folder)
    
    url = 'https://developer.apple.com/videos/play/wwdc{}/{}/'.format(year,session)
    print url
    
    file_name_path = '{}/{}'.format(folder,file_name(year,session))
    
    def script_content(url):

        header = {'User-Agent': random.choice(USER_AGENTS),}

        html = requests.get(url=url, headers=header).text

        soup = BeautifulSoup(html, 'lxml')

        ps = soup.find(class_='margin-bottom-small download-transcript').parent.find_all('p')
        re_text = '.*videos/wwdc/{}/{}(.*).pdf.*'.format(year,session)
        slides = soup.find('a', href=re.compile(re_text))['href']
        
        captions = []
        for p in ps:

            spans = p.find_all(class_='sentence')
            span_caption = ''
            for span in spans:
                span_caption += span.string

            span_caption = re.sub('[\r\n\t]', '', span_caption)
            captions.append(span_caption + '\n\n')

        return captions,slides

    def save_caption_to_file(captions):

        file = open(file_name_path,'w')             
        for caption in captions:
            file.write(caption.encode('utf8')) 
        file.close()

    # 爬取字幕
    content = script_content(url)
    captions = content[0]
    slides = content[1]
    
    # 保存字幕到txt文件中
    save_caption_to_file(captions)
    
    return file_name_path,slides

## download slides

将session的pdf文件一并下载下来。

In [111]:
def download_slides(url,year,session):

    r = requests.get(url)
    pdf_path = '{}/{}'.format(folder_path(year,session),file_name(year,session,'pdf'))
    
    try:
        with open(pdf_path, "wb") as code:
             code.write(r.content)
        print 'download sliders success'
    except Exception as e:
        print e

## main

爬取wwdc字幕并翻译的主调函数，传入year、session等待字幕下载、翻译完成即可。

In [63]:
def scrpit_wwdc_then_translate_save(year,session):
    
    script_wwdc_content = scrpit_wwdc(year,session)
    file_name_path = script_wwdc_content[0]
    slides_url = script_wwdc_content[1]
    
    translate_caption(file_name_path)
    download_slides(slides_url,year,session)
    
    print 'finished'

## example

In [110]:
scrpit_wwdc_then_translate_save(2018,407)

---  There is this folder!  ---
https://developer.apple.com/videos/play/wwdc2018/407/
translating:1/28
translating:2/28
translating:3/28
translating:4/28
translating:5/28
translating:6/28
translating:7/28
translating:8/28
translating:9/28
translating:10/28
translating:11/28
translating:12/28
translating:13/28
translating:14/28
translating:15/28
translating:16/28
translating:17/28
translating:18/28
translating:19/28
translating:20/28
translating:21/28
'ascii' codec can't decode byte 0xc3 in position 376: ordinal not in range(128)
translating:22/28
translating:23/28
translating:24/28
translating:25/28
translating:26/28
translating:27/28
translating:28/28
download sliders finished
finished


![image.png](./img/translate.png)